In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances
import warnings 
warnings.filterwarnings('ignore')
import pickle

: 

In [ ]:
# apload data
df2 = pd.read_csv("Dataset_Small_range.csv")
df1 = pd.read_csv("new_DATASET1 (1).csv")

In [ ]:
# Fusionner les deux ensembles de données en utilisant la colonne commune : Question_id
df = pd.merge(df1, df2, on='Question_id')
df.head()

,Question_id,Question_text,Score,Answers
0,1,ما هي العلاقة بين المغرب والدول الأوروبية خلال...,0.0,الدول الأوروبية
1,1,ما هي العلاقة بين المغرب والدول الأوروبية خلال...,0.0,الدول الأوروبية المغرب
2,1,ما هي العلاقة بين المغرب والدول الأوروبية خلال...,0.0,كانت العلاقة بين المغرب و الدول الأوروبية
3,1,ما هي العلاقة بين المغرب والدول الأوروبية خلال...,0.0,تميزت العلاقة بين المغرب والدول الأوروبية
4,1,ما هي العلاقة بين المغرب والدول الأوروبية خلال...,4.0,استعمار الدول الأوروبية المغرب


In [ ]:
import nltk
from nltk.tokenize import word_tokenize

In [ ]:
#Tokenisation
df['Answers'] = df['Answers'].apply(nltk.word_tokenize)

In [ ]:
#Drop stop words
from nltk.corpus import stopwords

stop_words = set(stopwords.words('Arabic'))
df['Answers'] = df['Answers'].apply(lambda x: [w for w in x if not w in stop_words])

In [ ]:
#Lemmatization & stemming
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
# Stem words in answers column
df['Answers'] = df['Answers'].apply(lambda x: [stemmer.stem(word) for word in x])

lemmatizer = WordNetLemmatizer()
# Lemmatize words in answers column
df['Answers'] = df['Answers'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

df.head()

,Question_id,Question_text,Score,Answers
0,1,ما هي العلاقة بين المغرب والدول الأوروبية خلال...,0.0,"[الدول, الأوروبية]"
1,1,ما هي العلاقة بين المغرب والدول الأوروبية خلال...,0.0,"[الدول, الأوروبية, المغرب]"
2,1,ما هي العلاقة بين المغرب والدول الأوروبية خلال...,0.0,"[كانت, العلاقة, المغرب, الدول, الأوروبية]"
3,1,ما هي العلاقة بين المغرب والدول الأوروبية خلال...,0.0,"[تميزت, العلاقة, المغرب, والدول, الأوروبية]"
4,1,ما هي العلاقة بين المغرب والدول الأوروبية خلال...,4.0,"[استعمار, الدول, الأوروبية, المغرب]"


In [ ]:
#Word2Vec
from gensim.models import Word2Vec
model = Word2Vec(df['Answers'], min_count=1)

In [ ]:
#Vectorise toutes les réponses
max_len = model.vector_size

W = np.zeros((len(df['Answers']), max_len))
for i, sentence in enumerate(df['Answers']):
    word_vectors = []
    for word in sentence:
        if word in model.wv:
            word_vectors.append(model.wv[word])
    if len(word_vectors) > 0:
        mean_vector = np.mean(word_vectors, axis=0)
        W[i, :] = mean_vector

In [ ]:
# Demander à l'utilisateur d'entrer une réponse et un id de question
question_id= 2
response = "تعاني العلاقة بين المغرب والجزائر من توترات وتحديات بسبب الصحراء"

In [ ]:
# Tokeniser la réponse entrée
response_tokens = nltk.word_tokenize(response)

# Supprimer les mots d'arrêt de la réponse
response_tokens = [w for w in response_tokens if not w in stop_words]

# Appliquer le stemming à la réponse
response_tokens = [stemmer.stem(word) for word in response_tokens]

# Appliquer la lemmatisation à la réponse
response_tokens = [lemmatizer.lemmatize(word) for word in response_tokens]

# Vectoriser la réponse en utilisant le modèle Word2Vec
response_vector = np.mean([model.wv[word] for word in response_tokens if word in model.wv], axis=0)

response_vector
response_tokens

['تعاني',
 'العلاقة',
 'المغرب',
 'والجزائر',
 'توترات',
 'وتحديات',
 'بسبب',
 'الصحراء']

In [ ]:
def find_existing_answers_vectors(df, question_id, W):

    if question_id in df['Question_id'].values:
        # Trouver tous les indices correspondants à l'ID de la question
        question_indices = df.index[df['Question_id'] == question_id].tolist()

        # Extraire les vecteurs de réponse correspondants à partir de la matrice W
        existing_answers_vectors = []
        for i in question_indices:
            existing_answers_vectors.append(W[i])
    else:
        print("L'ID de la question que vous avez fourni est incorrect")
        existing_answers_vectors = None
    existing_answers_vectors = np.array(existing_answers_vectors)
    return existing_answers_vectors

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor

existing_answers_vectors = find_existing_answers_vectors(df, question_id, W)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(existing_answers_vectors, df[(df['Question_id'] == question_id)]['Score'], test_size=0.2, random_state=42)

# Scale input data to [0, 1] range
#scaler = MinMaxScaler()
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)

# Create and train the MLP regression model
mlp_model = MLPRegressor(hidden_layer_sizes=(100, 100), activation='relu', solver='adam', random_state=42)
mlp_model.fit(X_train, y_train)

# Reshape response vector to have shape (1, n)
response_vector_reshaped = response_vector.reshape(1, -1)

# Predict the score using the MLP regression model
score_prediction = mlp_model.predict(response_vector_reshaped)

# Print the predicted score
print("Predicted score:", score_prediction[0])


Predicted score: 2.779742758562498


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR

existing_answers_vectors = find_existing_answers_vectors(df, question_id, W)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(existing_answers_vectors, df[(df['Question_id'] == question_id)]['Score'], test_size=0.2, random_state=42)

# Scale input data to [0, 1] range
#scaler = MinMaxScaler()
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)

# Create and train the SVR model
svr_model = SVR(kernel='rbf', C=1.0, epsilon=0.1)
svr_model.fit(X_train, y_train)

# Reshape response vector to have shape (1, n)
response_vector_reshaped = response_vector.reshape(1, -1)

# Predict the score using the SVR model
score_prediction = svr_model.predict(response_vector_reshaped)

# Print the predicted score
print("Predicted score:", score_prediction[0])


Predicted score: 2.8498295352896523


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

existing_answers_vectors = find_existing_answers_vectors(df, question_id, W)

# Initialize an empty array to store the cosine similarities
cosine_similarities = []

# Loop over the existing answer vectors and compute cosine similarity with the response vector
for answer_vector in existing_answers_vectors:
    cosine_similarity_score = cosine_similarity(response_vector.reshape(1, -1), answer_vector.reshape(1, -1))
    cosine_similarities.append(cosine_similarity_score[0][0])

# Convert the cosine similarities to a NumPy array
cosine_similarities = np.array(cosine_similarities)

# Concatenate the cosine similarities with the existing answer vectors
combined_features = np.column_stack((existing_answers_vectors, cosine_similarities))

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, df[df['Question_id'] == question_id]['Score'], test_size=0.2, random_state=42)

# Scale input data to [0, 1] range
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and train the Random Forest Regression model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Reshape response vector to have the same shape as existing answer vectors
response_vector_reshaped = np.tile(response_vector, (existing_answers_vectors.shape[0], 1))

# Initialize an empty array to store the response cosine similarities
response_cosine_similarities = []

# Loop over the existing answer vectors and compute cosine similarity with the response vector
for answer_vector in existing_answers_vectors:
    response_cosine_similarity_score = cosine_similarity(response_vector_reshaped, answer_vector.reshape(1, -1))
    response_cosine_similarities.append(response_cosine_similarity_score[0][0])

# Convert the response cosine similarities to a NumPy array
response_cosine_similarities = np.array(response_cosine_similarities)

# Reshape response_cosine_similarities to match the shape of response_vector_reshaped
response_cosine_similarities = response_cosine_similarities.reshape(-1, 1)

# Concatenate the cosine similarities with the response vector
response_combined_features = np.column_stack((response_vector_reshaped, response_cosine_similarities))

# Scale the response vector
response_combined_features_scaled = scaler.transform(response_combined_features)

# Predict the score using the Random Forest Regression model
score_prediction = rf_model.predict(response_combined_features_scaled)

# Print the predicted score
print("Predicted score:", score_prediction[0])


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

existing_answers_vectors = find_existing_answers_vectors(df, question_id, W)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(existing_answers_vectors, df[(df['Question_id'] == question_id)]['Score'], test_size=0.2, random_state=42)

# Scale input data to [0, 1] range
#scaler = MinMaxScaler()
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)

# Create and train the Random Forest Regression model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Reshape response vector to have shape (1, n)
response_vector_reshaped = response_vector.reshape(1, -1)

# Predict the score using the Random Forest Regression model
score_prediction = rf_model.predict(response_vector_reshaped)

# Print the predicted score
print("Predicted score:", score_prediction[0])


Predicted score: 3.06


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor

existing_answers_vectors = find_existing_answers_vectors(df, question_id, W)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(existing_answers_vectors, df[(df['Question_id'] == question_id)]['Score'], test_size=0.2, random_state=42)

# Scale input data to [0, 1] range
#scaler = MinMaxScaler()
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)

# Create and train the Gradient Boosted Regression model
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

# Reshape response vector to have shape (1, n)
response_vector_reshaped = response_vector.reshape(1, -1)

# Predict the score using the Gradient Boosted Regression model
score_prediction = gb_model.predict(response_vector_reshaped)
print("Predicted score:", score_prediction[0])

Predicted score: 3.9772640260739336


In [ ]:
# -----------------------------------------Pickling the model -------------------------------------------------------
pickle.dump(gb_model,open("ModelPerfect.pkl","wb"))
loaded_model = pickle.load(open("ModelPerfect.pkl","rb"))
result = loaded_model.predict(response_vector_reshaped)
print("Result score:", result[0])

Result score: 3.9772640260739336


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

existing_answers_vectors = find_existing_answers_vectors(df, question_id, W)

# Initialize an empty array to store the cosine similarities
cosine_similarities = []

# Loop over the existing answer vectors and compute cosine similarity with the response vector
for answer_vector in existing_answers_vectors:
    cosine_similarity_score = cosine_similarity(response_vector.reshape(1, -1), answer_vector.reshape(1, -1))
    cosine_similarities.append(cosine_similarity_score[0][0])

# Convert the cosine similarities to a NumPy array
cosine_similarities = np.array(cosine_similarities)

# Concatenate the cosine similarities with the existing answer vectors
combined_features = np.column_stack((existing_answers_vectors, cosine_similarities))

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, df[df['Question_id'] == question_id]['Score'], test_size=0.2, random_state=42)

# Scale input data to [0, 1] range
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and train the GradientBoostingRegressor model
gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train_scaled, y_train)

# Reshape response vector to have the same shape as existing answer vectors
response_vector_reshaped = np.tile(response_vector, (existing_answers_vectors.shape[0], 1))

# Initialize an empty array to store the response cosine similarities
response_cosine_similarities = []

# Loop over the existing answer vectors and compute cosine similarity with the response vector
for answer_vector in existing_answers_vectors:
    response_cosine_similarity_score = cosine_similarity(response_vector_reshaped, answer_vector.reshape(1, -1))
    response_cosine_similarities.append(response_cosine_similarity_score[0][0])

# Convert the response cosine similarities to a NumPy array
response_cosine_similarities = np.array(response_cosine_similarities)

# Reshape response_cosine_similarities to match the shape of response_vector_reshaped
response_cosine_similarities = response_cosine_similarities.reshape(-1, 1)

# Concatenate the cosine similarities with the response vector
response_combined_features = np.column_stack((response_vector_reshaped, response_cosine_similarities))

# Scale the response vector
response_combined_features_scaled = scaler.transform(response_combined_features)

# Predict the score using the GradientBoostingRegressor model
score_prediction = gb_model.predict(response_combined_features_scaled)

# Print the predicted score
print("Predicted score:", score_prediction[0])


Predicted score: 2.299124748729821
